In [1]:
# CE 295 - Energy Systems and Control
# Final Project: Model Predictive Control and the Optimal Power Flow Problem in the IEEE 39-bus Test Feededr
# Authors (alphabetical): Carla Becker, Hannah Davalos, Jean-Luc Lupien, John Schafer, Keyi Yang
# Adapted from code provided by Prof. Daniel B. Arnold

from cvxpy import *
import data_processing as dp
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import pandas as pd

In [2]:
# Import data for IEEE 39-bus Test Feeder (plus 8 battery nodes)

# Define transformer resistance and reactance (not provided by test feeder)
xformer_r = 0.0001
xformer_x = 0.0015

# Adjacency matrix; assumes power can flow in two directions on all lines
A_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='A matrix')
A_df = A_df.iloc[:,1:] # remove the column labels
A    = A_df.values     # convert to a numpy array

# Resistance matrix
r_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='r matrix')
r_df = r_df.iloc[:,1:] # remove the column labels
r    = r_df.values     # convert to a numpy array

# Reactance matrix
x_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='x matrix')
x_df = x_df.iloc[:,1:] # remove the column labels
x    = x_df.values     # convert to a numpy array

# Get parents vector (matrix??)
rho_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='rho')
rho_df = rho_df.iloc[:,1:] # remove the column labels

# TODO how to allow for multiple parents?
future_rho = {}
for node in rho_df.columns:
    parents = np.array(rho_df[node])
    nonzero_parents = parents[parents != 0]
    key = str(node - 1) # account for indexing from 0
    future_rho[key] = nonzero_parents.tolist()

rho = rho_df.values[0,:] # TEMPORARY !! must figure out how to do multiple parents

# Number of nodes
num_nodes = A.shape[0]

# Diesel nodes
diesel_nodes = np.array([31])
diesel_nodes = diesel_nodes - 1 # adjust for indexing from 0
max_diesel_power = 10 # TODO set sensible value
diesel_cost = 100 # TODO set sensible value

# Battery energy storage nodes
BESS_nodes = np.array([40, 41, 42, 43, 44, 45, 46, 47])
BESS_nodes = BESS_nodes - 1 # adjust for indexing from 0
max_battery_power = 10 # TODO set sensible value
battery_cost = 0

# Wind nodes
wind_nodes = np.array([30, 37, 38])
wind_nodes = wind_nodes - 1 # adjust for indexing from 0
max_wind_power = 10 # TODO set sensible value
wind_cost = 0

# Solar nodes
solar_nodes = np.array([32, 33, 34, 35, 36])
solar_nodes = solar_nodes - 1 # adjust for indexing from 0
max_solar_power = 10 # TODO set sensible value
solar_cost = 0

# Consumer nodes
consumer_nodes = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 39])
consumer_nodes = consumer_nodes - 1 # adjust for indexing from 0

In [17]:
# Read in daily generation and load data
pv_directory   = 'PV Generation Data'
load_directory = 'Building Load Data'
wind_directory = 'Wind Generation Data'

# ALREADY GENERATED, takes 5 minutes to generate again so keep commented out
#dp.generate_json_from_pv_data(pv_directory) 
#dp.generate_json_from_bldg_data(load_directory) 
#dp.generate_json_from_wind_data(wind_directory) 

with open(os.path.join(pv_directory, 'pv_data.json'), 'r') as json_file:
    pv_dict = json.load(json_file)

for key in pv_dict.keys(): # get data from first key only (CAPTL_WF)
    solar_data = np.array(pv_dict[key])
    break

# for key, value in pv_dict.items():
    # print(key, value['0'])
    # break

with open(os.path.join(load_directory, 'real_data.json'), 'r') as json_file:
    real_load_dict = json.load(json_file)

with open(os.path.join(load_directory, 'reactive_data.json'), 'r') as json_file:
    reactive_load_dict = json.load(json_file)

with open(os.path.join(wind_directory, 'wind_data.json'), 'r') as json_file:
    wind_dict = json.load(json_file)

# There are 67 PV panels
# There are 27 buildings
# There are 22 wind turbines

# There are 427 days of data for each entity
# Each data has 96 data points: one measurement every 15 minutes

PowellPV0 96


In [36]:
# Get mean and std for each time period for solar power

TimeValues_Solar = []
numDays = 427
numTimes = 96
Means_Solar = []
StDevs_Solar = []

for time in range(numTimes):
    TimeValues_Solar.append([])

# print(TimeValues)

for key, value in pv_dict.items():
    for day in range(numDays):
        for time in range(numTimes):
            TimeValues_Solar[time].append(value[str(day)][time])

for timeList in TimeValues_Solar:
    Means_Solar.append(np.mean(timeList))
    StDevs_Solar.append(np.std(timeList))
    

# print(len(TimeValues[1]))
# print(Means)
# print(StDevs)

28609
[0.0012572300164583005, 0.001383063309975727, 0.0018898684739547407, 0.002138900955518824, 0.0024315612076097237, 0.002849862456086275, 0.0034116159566406606, 0.00418558917503267, 0.005355491429078891, 0.006947779367780896, 0.009013095454520854, 0.01151393561876634, 0.014833688219044168, 0.018754295349179884, 0.023414481155816166, 0.02868423884457657, 0.034603929697243095, 0.04135353006237712, 0.04856930782693791, 0.05745261837234709, 0.06763743801135119, 0.0795498744393824, 0.09328634463874777, 0.10915474937671442, 0.12762784225637475, 0.14827256651913603, 0.17184528063839513, 0.1980723670466442, 0.22683581358580315, 0.2573503410091594, 0.28932061746726584, 0.3216162467191643, 0.35435135221867975, 0.3877220720557753, 0.420090781321993, 0.45081565873813056, 0.4806348224523683, 0.5077839040516852, 0.5299547483887644, 0.5501870973441402, 0.565510102786728, 0.5789453478948059, 0.5886748769828403, 0.5942734562693482, 0.5964429475865766, 0.5963629127590901, 0.5912014375200303, 0.58420

In [43]:
# Get mean and std for each time period for wind power

TimeValues_Wind = []
numDays = 427
numTimes = 96
Means_Wind = []
StDevs_Wind = []

for time in range(numTimes):
    TimeValues_Wind.append([])

# print(TimeValues)

for key, value in wind_dict.items():
    for day in range(numDays):
        for time in range(numTimes):
            TimeValues_Wind[time].append(value[str(day)][time])

for timeList in TimeValues_Wind:
    Means_Wind.append(np.mean(timeList))
    StDevs_Wind.append(np.std(timeList))

# print(TimeValues_Wind)

# print(len(TimeValues_Wind[0]))

# print(Means_Wind)
# print(StDevs_Wind)

[0.31159362413343317, 0.3113351583877669, 0.31156478411236294, 0.3119723607279309, 0.31202699307138243, 0.3121433081014509, 0.3126708322761463, 0.3131085353390454, 0.3123394029115253, 0.3122801567977653, 0.31177942934629793, 0.3123025321417954, 0.3128727966549993, 0.3123525715082702, 0.3127351717176362, 0.3125121710446692, 0.3125286295663385, 0.3128205158286188, 0.312869706968216, 0.31234649306542067, 0.3117334862551023, 0.3124130641231, 0.3129978942973854, 0.31223559086083813, 0.31242837118664296, 0.312376588918881, 0.31219135269001996, 0.3127494449461271, 0.3128142922529055, 0.31388374709214334, 0.3133927873452277, 0.3142190368348219, 0.3131164916483549, 0.31237730648923207, 0.31182146230976476, 0.3103019271916899, 0.3091457543192031, 0.3086140184379069, 0.3084881926162219, 0.3079034559692919, 0.30813806641205654, 0.3090245774112058, 0.3099442222675857, 0.3105283719664466, 0.3107640204764617, 0.3112224240073816, 0.3125657349463235, 0.31487257087303516, 0.3153512060389632, 0.314584822

In [4]:
## 39 Node IEEE Test Feeder Parameters

### Node (aka Bus) Data
# l_j^P: Active power consumption [MW]
cons_real_power = real_load_dict

# l_j^Q: Reactive power consumption [MVAr]
cons_reactive_power = reactive_load_dict

# s_j,max: Maximal generating power [MW]
max_apparent_power = np.zeros(num_nodes)
max_apparent_power[diesel_nodes] = max_diesel_power
max_apparent_power[BESS_nodes]   = max_battery_power
max_apparent_power[wind_nodes]   = max_wind_power
max_apparent_power[solar_nodes]  = max_solar_power
max_apparent_power = max_apparent_power.reshape(1, num_nodes)

# c_j: Marginal generation cost [USD/MW]
c = np.zeros(num_nodes)
c[diesel_nodes] = diesel_cost
c[BESS_nodes]   = battery_cost
c[wind_nodes]   = wind_cost
c[solar_nodes]  = solar_cost
c = c.reshape(num_nodes, 1)

# V_min, V_max: Minimum and maximum nodal voltages [V]
min_voltage = 0.95
max_voltage = 1.05

# I_max_ij: Maximal line current [p.u.]
I_max = A * 0.01

### Set Data
# List of node indices
j_idx = np.arange(num_nodes) # TODO necessary?

In [5]:
# Organize consumed real and reactive power into numpy arrays
# Only load ** 1 ** day of data (horizon is 1 day, 96 points)
num_bldgs = len(real_load_dict.keys())
real_power_consumed     = np.zeros((len(consumer_nodes), 96))
reactive_power_consumed = np.zeros((len(consumer_nodes), 96))

# For first 27 consumer nodes (only 27 buildings)
for node in range(num_bldgs):
    for b, bldg in enumerate(real_load_dict.keys()):
        if node % num_bldgs == b:
            real_power_consumed[node, :]     = real_load_dict[bldg]['0'] # ['0'] because only 1 day
            reactive_power_consumed[node, :] = reactive_load_dict[bldg]['0']

# For remaining consumer nodes, repeat some of the buildings
for node in range(num_bldgs, len(consumer_nodes)):
    for b, bldg in enumerate(real_load_dict.keys()):
        if node % num_bldgs == b:
            real_power_consumed[node, :]     = real_load_dict[bldg]['0']
            reactive_power_consumed[node, :] = reactive_load_dict[bldg]['0']

# Now, only load these values into the consumer nodes, have zeros for all other nodes
cons_real_power     = np.zeros((num_nodes, 96))
cons_reactive_power = np.zeros((num_nodes, 96))

cons_real_power[consumer_nodes, :]     = real_power_consumed
cons_reactive_power[consumer_nodes, :] = reactive_power_consumed

In [6]:
def cvx_optim(cons_real_power, cons_reactive_power, max_apparent_power, min_voltage, max_voltage, BESS_nodes, wind_nodes, solar_nodes, diesel_nodes):

    num_nodes = cons_real_power.shape[0]
    horizon   = cons_real_power.shape[1]
    renew_nodes = np.concatenate([wind_nodes, solar_nodes])
    
    # Define optimization variables for generated power
    gen_real_power       = Variable((num_nodes, horizon))
    gen_reactive_power   = Variable((num_nodes, horizon))
    gen_apparent_power   = Variable((num_nodes, horizon))

    # Define optimization variables for active line power
    # index 0 is the time step, index 1 is node 1, index 2 is node 2
    line_real_power      = [Variable((num_nodes, num_nodes)) for t in range(horizon)] 
    line_reactive_power  = [Variable((num_nodes, num_nodes)) for t in range(horizon)]
    line_complex_current = [Variable((num_nodes, num_nodes)) for t in range(horizon)]

    # Line voltage decision variable
    line_voltages = Variable((num_nodes, horizon))
    
    # Decision variable for robust optimization
    # TODO read from PV and wind data instead ??
    sigma_A = Variable((len(renew_nodes), horizon)) 

    '''
    TODO properly integrate BESS
    # Define optimization variables for battery energy storage system (BESS)
    BESS_energy          = Variable((num_nodes, horizon))
    BESS_chrg_dis        = Variable((num_nodes, horizon))

    # BESS Parameters
    timestep=.25 # assuming 15min timestep
    eta=.95 #(charging /discharging efficiency)
    energy_min=0 #minimun energy level of BESS, need to define for every node seperately?
    #100 max energy level of BESS, in kWh NEED TO SCALE
    energy_max=np.array([0, 0, 0,0, 100, 100, 0, 0, 0, 0, 0]) #placeholder, need actual nodes that have BESS, needs to have 39 nodes
    
    ramp_max= 100 / timestep #1C, can discharge/charge 1 full capacity every hour, assuming timestep is in hours
    #initial_energy=np.zeros(BESS_nodes) #place holder
    #where to I specify which nodes are BESS?

    # BESS initial state
    #constraints = [BESS_energy[:, 0] == initial_energy]  # initial_energy needs to be defined
    '''

    # Define objective function
    objective =  Minimize(sum(sum(c.T @ gen_apparent_power)))

    # Define constraints
    # Apparent Power Limits
    constraints = [gen_apparent_power[:, t] <= np.squeeze(max_apparent_power) for t in range(horizon)]

    # Constants for robust optimization constants
    a = np.array([-1.25, 1])
    bar_a = np.tile(a, len(renew_nodes))

    e_array = np.array((0.25, 0))
    E = np.diag(np.tile(e_array, len(renew_nodes)))

    # Loop over every time step in the horizon, loop over each nodes in each time step
    for t in range(horizon):

        print(f't: {t}')

        # Define robust optimization decision variables for intermittent energy sources
        y = vstack([sigma_A[0, t], gen_apparent_power[renew_nodes[0]][t]])
        for i in range(1, len(renew_nodes)):
            y = vstack([y, vstack([sigma_A[i, t], gen_apparent_power[renew_nodes[i]][t]])])

        # Robust Optimization Constraints
        constraints += [bar_a @ y + norm(E @ y) <= 0]
        constraints += [sigma_A[:, t] >= 0, sigma_A[:, t] <= 1]

        # Diesel node constraints
        for node in diesel_nodes:
            diesel_node = int(node)

            # Boundary condition for power line flows
            constraints += [line_real_power[t][diesel_node][diesel_node] == 0]
            constraints += [line_reactive_power[t][diesel_node][diesel_node] == 0]

            # Boundary condition for squared line current
            constraints += [line_complex_current[t][diesel_node][diesel_node] == 0]

            # Fix diesel node voltage to be 1 "per unit" (p.u.)
            constraints += [line_voltages[diesel_node][t] == 1]

        for jj in j_idx: # TODO test for more nodes j_idx

            print(f'jj: {jj}')
            
            '''
            TODO properly integrate BESS
            #BESS static Constraints
            constraints +=[energy_min<=BESS_energy[jj, t], BESS_energy[jj, t] <= energy_max[jj]] #every battery node has energy limits for every time

            if t < horizon - 1:
                # Ramp rate constraints
                constraints += [BESS_chrg_dis[jj, t + 1] - BESS_chrg_dis[jj, t] <= ramp_max,
                                BESS_chrg_dis[jj, t] - BESS_chrg_dis[jj, t + 1] <= ramp_max]

                # BESS energy dynamics
                constraints += [BESS_energy[jj, t + 1] == BESS_energy[jj, t] + timestep * eta * BESS_chrg_dis[jj, t]]
            '''

            # Nodal voltage limits, not dependent on t
            constraints += [min_voltage**2 <= line_voltages[jj,t]]
            constraints += [line_voltages[jj,t] <= max_voltage**2]

            # Non-negative power generation
            constraints += [gen_real_power[jj, t] >= 0]
            constraints += [gen_reactive_power[jj, t] >= 0]

            # Parent node, i = rho(j)
            ii = rho[jj] # TODO change to work for multiple parents, see the variable future_rho

            # Squared line current limits
            constraints += [line_complex_current[t][ii][jj] <= (I_max[ii][jj])**2] 

            # Line Power Flows
            constraints += [line_real_power[t][ii][jj]     == (cons_reactive_power[jj][t] - gen_apparent_power[jj][t]) + r[ii][jj]*line_complex_current[t][ii][jj] + A[jj]@line_real_power[jj].T]
            constraints += [line_reactive_power[t][ii][jj] == (cons_reactive_power[jj][t] - gen_apparent_power[jj][t]) + x[ii][jj]*line_complex_current[t][ii][jj] + A[jj]@line_real_power[jj].T]

            # Nodal voltage
            constraints += [line_voltages[jj][t] == line_voltages[ii][t] + ((r[ii][jj])**2 + (x[ii][jj])**2)*line_complex_current[ii][jj] - 2*(r[ii][jj]*line_real_power[ii][jj]+x[ii][jj]*line_reactive_power[ii][jj])]
            
            # Squared current magnitude on lines
            constraints += [line_complex_current[t][ii][jj] >= quad_over_lin(vstack([line_real_power[t][ii][jj], line_reactive_power[t][ii][jj]]), line_voltages[jj][t])]

            # Compute apparent power from active & reactive power
            constraints += [norm(vstack([gen_real_power[jj][t], gen_reactive_power[jj][t]])) <= gen_apparent_power[jj][t]]

    # Define problem and solve
    prob = Problem(objective, constraints)
    prob.solve()

    # Output Results
    print(prob.status)
    print(f"Minimum Generating Cost: {prob.value} USD")

    print(" ")
    print(f"Node 30 [Diesel]: real power = {gen_real_power[30].value} MW | reactive power = {gen_reactive_power[30].value} MVAr | apparent power = {gen_apparent_power[30].value} MVA")
    
    print(" ")
    for node in renew_nodes:
        print(f"Node {node} [Renewable]: real power = {gen_real_power[node].value} MW | reactive power = {gen_reactive_power[node].value} MVAr | apparent power = {gen_apparent_power[node].value} MVA")
    
    print(" ")
    #print(f"Total active real power:     {np.sum(line_real_power)} MW consumed | {np.sum(gen_real_power)} MW generated")
    #print(f"Total active reactive power: {np.sum(line_reactive_power)} MW consumed | {np.sum(gen_reactive_power)} MW generated")
    

In [7]:
# Test csv_optim function
cvx_optim(cons_real_power, cons_reactive_power, max_apparent_power, min_voltage, max_voltage, BESS_nodes, wind_nodes, solar_nodes, diesel_nodes)

t: 0
jj: 0
jj: 1
jj: 2
jj: 3
jj: 4
jj: 5
jj: 6
jj: 7
jj: 8
jj: 9
jj: 10
jj: 11
jj: 12
jj: 13
jj: 14
jj: 15
jj: 16
jj: 17
jj: 18
jj: 19
jj: 20
jj: 21
jj: 22
jj: 23
jj: 24
jj: 25
jj: 26
jj: 27
jj: 28
jj: 29
jj: 30
jj: 31
jj: 32
jj: 33
jj: 34
jj: 35
jj: 36
jj: 37
jj: 38
jj: 39
jj: 40
jj: 41
jj: 42
jj: 43
jj: 44
jj: 45
jj: 46
t: 1
jj: 0
jj: 1
jj: 2
jj: 3
jj: 4
jj: 5
jj: 6
jj: 7
jj: 8
jj: 9
jj: 10
jj: 11
jj: 12
jj: 13
jj: 14
jj: 15
jj: 16
jj: 17
jj: 18
jj: 19
jj: 20
jj: 21
jj: 22
jj: 23
jj: 24
jj: 25
jj: 26
jj: 27
jj: 28
jj: 29
jj: 30
jj: 31
jj: 32
jj: 33
jj: 34
jj: 35
jj: 36
jj: 37
jj: 38
jj: 39
jj: 40
jj: 41
jj: 42
jj: 43
jj: 44
jj: 45
jj: 46
t: 2
jj: 0
jj: 1
jj: 2
jj: 3
jj: 4
jj: 5
jj: 6
jj: 7
jj: 8
jj: 9
jj: 10
jj: 11
jj: 12
jj: 13
jj: 14
jj: 15
jj: 16
jj: 17
jj: 18
jj: 19
jj: 20
jj: 21
jj: 22
jj: 23
jj: 24
jj: 25
jj: 26
jj: 27
jj: 28
jj: 29
jj: 30
jj: 31
jj: 32
jj: 33
jj: 34
jj: 35
jj: 36
jj: 37
jj: 38
jj: 39
jj: 40
jj: 41
jj: 42
jj: 43
jj: 44
jj: 45
jj: 46
t: 3
jj: 0
jj: 1
jj: 2
jj: 3

/Users/hdavalos/Library/Python/3.9/lib/python/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


infeasible_inaccurate
Minimum Generating Cost: inf USD
 
Node 30 [Diesel]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
 
Node 29 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 36 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 37 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 31 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 32 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 33 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 34 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 35 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
 
